In [13]:
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from pandas import Series

In [14]:
url = "http://www.omdbapi.com/?apikey=trilogy&t="

In [15]:
#read in to CSV File
movies_df = pd.read_csv("2013-2017 Box Office Data.csv")


In [16]:
#Clean up Movie Titles, removing (2015) from Cinderalla, (2016) Jungle Book, (2017) Beauty and the Beast
movies_df = movies_df.reset_index(level=None, drop = False, inplace = False, col_level = 0, col_fill = "")
s = pd.DataFrame(movies_df["Movie Title"])
s1 = s.reset_index(level=None, drop = False, inplace = False, col_level = 0, col_fill = "")
s2 = s1["Movie Title"].str.split("(").apply(Series, 1)
s3 = s2.reset_index(level=None, drop = False, inplace = False, col_level = 0, col_fill = "")

In [18]:
#merge the dataframes
movies_merge = movies_df.merge(s3, how='inner', on="index", left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)
movies_merge

,index,Rank,Movie Title,Studio,Total Gross,Theaters,Opening,Theaters.1,Open,Close,Year,0,1
0,0,1,The Hunger Games: Catching Fire,LGF,"$424,668,047","4,163","$158,074,286","4,163",22-Nov,3-Apr,2013,The Hunger Games: Catching Fire,NaN
1,1,2,Iron Man 3,BV,"$409,013,994","4,253","$174,144,585","4,253",3-May,12-Sep,2013,Iron Man 3,NaN
2,2,3,Frozen,BV,"$400,738,009","3,742","$243,390",1,22-Nov,17-Jul,2013,Frozen,NaN
3,3,4,Despicable Me 2,Uni.,"$368,061,265","4,003","$83,517,315","3,997",3-Jul,16-Jan,2013,Despicable Me 2,NaN
4,4,5,Man of Steel,WB,"$291,045,518","4,207","$116,619,362","4,207",14-Jun,19-Sep,2013,Man of Steel,NaN
5,5,6,Gravity,WB,"$274,092,705","3,820","$55,785,112","3,575",4-Oct,8-May,2013,Gravity,NaN
6,6,7,Monsters University,BV,"$268,492,764","4,004","$82,429,469","4,004",21-Jun,19-Dec,2013,Monsters University,NaN
7,7,8,The Hobbit: The Desolation of Smaug,WB (NL),"$258,366,855","3,928","$73,645,197","3,903",13-Dec,17-Apr,2013,The Hobbit: The Desolation of Smaug,NaN
8,8,9,Fast & Furious 6,Uni.,"$238,679,850","3,771","$97,375,245","3,658",24-May,5-Sep,2013,Fast & Furious 6,NaN
9,9,10,Oz The Great and Powerful,BV,"$234,911,825","3,912","$79,110,453","3,912",8-Mar,18-Jul,2013,Oz The Great and Powerful,NaN


In [19]:
#delete excessive columns 
del movies_merge[1]
del movies_merge["index"]
del movies_merge["Theaters.1"]
movies_merge = movies_merge.rename(columns = {0:"Movie Title (loop)"})
movies_df = movies_merge[["Movie Title", "Movie Title (loop)", "Year", "Studio","Total Gross","Theaters", "Opening", "Open", "Close"]]



In [20]:
#Add the Additional Columns
movies_df["Rated"] = ""
movies_df["Runtime"]= ""
movies_df["Director"]=""
movies_df["Actor 1"] = ""
movies_df["Actor 2"] = ""
movies_df["Rotten Tomatoes"] = ""
movies_df["Metacritic"] = ""
movies_df["IMDB Rating"] = ""
movies_df["Production Studio"] =""
movies_df.head()




,Movie Title,Movie Title (loop),Year,Studio,Total Gross,Theaters,Opening,Open,Close,Rated,Runtime,Director,Actor 1,Actor 2,Rotten Tomatoes,Metacritic,IMDB Rating,Production Studio
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,2013,LGF,"$424,668,047","4,163","$158,074,286",22-Nov,3-Apr,,,,,,,,,
1,Iron Man 3,Iron Man 3,2013,BV,"$409,013,994","4,253","$174,144,585",3-May,12-Sep,,,,,,,,,
2,Frozen,Frozen,2013,BV,"$400,738,009","3,742","$243,390",22-Nov,17-Jul,,,,,,,,,
3,Despicable Me 2,Despicable Me 2,2013,Uni.,"$368,061,265","4,003","$83,517,315",3-Jul,16-Jan,,,,,,,,,
4,Man of Steel,Man of Steel,2013,WB,"$291,045,518","4,207","$116,619,362",14-Jun,19-Sep,,,,,,,,,


In [21]:
for index, row in movies_df.iterrows():
    base_url = "http://www.omdbapi.com/?apikey=trilogy"
    params = {
        "t": row["Movie Title (loop)"],
        "y": row["Year"]
    }

    # make request and print url
    response = requests.get(base_url, params=params).json()
    
    
    try:
        movies_df.set_value(index, "Rated", response["Rated"])
        movies_df.set_value(index, "Runtime", response["Runtime"])
        movies_df.set_value(index, "Director", response["Director"])
        movies_df.set_value(index, "Actor 1", response["Actors"])
        movies_df.set_value(index, "Actor 2", response["Actors"])
        movies_df.set_value(index, "Rotten Tomatoes", response["Ratings"][1]["Value"])
        movies_df.set_value(index, "Metacritic", response["Ratings"][2]["Value"])
        movies_df.set_value(index, "IMDB Rating", response["Ratings"][0]["Value"])
        movies_df.set_value(index, "Production Studio", response["Production"])
            
        
    except (KeyError, IndexError):
        print(f" {movie} Missing field/result... skipping.")
    
   
    
    

In [22]:
#Clean data take out $ and commas to convert to integer
movies_df['Total Gross'] = movies_df['Total Gross'].str.replace('$','')
movies_df['Total Gross'] = movies_df['Total Gross'].str.replace(',','')
movies_df['Total Gross'] = pd.to_numeric(movies_df['Total Gross'])

movies_df['Opening'] = movies_df['Opening'].str.replace('$','')
movies_df['Opening'] = movies_df['Opening'].str.replace(',','')
movies_df['Opening'] = pd.to_numeric(movies_df['Opening'])

movies_df['Runtime'] = movies_df['Runtime'].str.replace('min','')
movies_df['Runtime'] = pd.to_numeric(movies_df['Runtime'])
movies_df = movies_df.rename(columns = {"Runtime":"Runtime (Minutes)"})

movies_df['Rotten Tomatoes'] = movies_df['Rotten Tomatoes'].str.replace('%','')
movies_df['Rotten Tomatoes'] = pd.to_numeric(movies_df['Rotten Tomatoes'])
movies_df = movies_df.rename(columns = {"Rotten Tomatoes":"Rotten Tomatoes (%)"})


In [23]:
for index, row in movies_df.iterrows():
    integer1 = int(row['Metacritic'].split('/')[0])
    integer2 = int(row['Metacritic'].split('/')[1])
    movies_df.set_value(index, "Metacritic", (integer1/integer2)*100)
    movies_df.set_value(index, "IMDB Rating", (float(row['IMDB Rating'].split('/')[0]))*10)
    movies_df.set_value(index, "Actor 1", str(row["Actor 1"].split(",")[0]))
    movies_df.set_value(index, "Actor 2", str(row["Actor 2"].split(",")[1]))
    movies_df.set_value(index, "Director", str(row["Director"].split(",")[0]))
  



In [24]:
movies_df.to_csv("output/movies_data.csv", encoding = "utf-8")